# Leser inn bibliotek og genererer testdata

## Leser inn bibliotek og setter opp spark session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [ ]:
myspark = SparkSession.builder.getOrCreate()

In [ ]:
%run ../../ssb_sparktools/editing/controls.py

## Lager testdata og kodelistedata

#### Testdata

In [ ]:
testdata_schema = StructType(
    [
        StructField("identifikator", StringType(), False),
        StructField("numbvar", LongType(), True),
        StructField("boolvar", BooleanType(), True),
        StructField("varenummer", StringType(), True),
    ]
)
testdata_raw = [
    ("id1", 1, True, "#001"),
    ("id2", 2, False, "#002"),
    ("id3", None, False, "#003"),
    ("id4", 4, None, "#004"),
    ("id5", 4, True, "#005"),
]
testdata = myspark.createDataFrame(testdata_raw, testdata_schema)

#### Kodelistedata

In [ ]:
testdata_kodeliste_schema = StructType(
    [
        StructField("varenummer", StringType(), True),
        StructField("frukt", StringType(), True),
    ]
)
testdata_kodeliste_raw = [
    ("#001", "Eple"),
    ("#002", "Banan"),
    ("#003", "Kiwi"),
    ("#004", "Appelsin"),
    ("#006", "Druer"),
]
testdata_kodeliste = myspark.createDataFrame(
    testdata_kodeliste_raw, testdata_kodeliste_schema
)

#### Skriver ut testdata som er laget

In [ ]:
testdata.toPandas()

In [ ]:
testdata_kodeliste.toPandas()

# Test av funksjonen *listcode_check*

In [ ]:
kodeliste = testdata_kodeliste.select("varenummer")
sjekk, sjekkdf = listcode_check(testdata, "varenummer", kodeliste)
sjekkdf.toPandas()

In [ ]:
sjekk

# Test av funksjon compare_dimdf

In [ ]:
compare_dimdf(testdata, testdata)

In [ ]:
nytestdata = testdata.drop("numbvar")
compare_dimdf(testdata, nytestdata)

In [ ]:
nytestdata = testdata.filter(F.col("identifikator") != "id4")
compare_dimdf(testdata, nytestdata)

In [ ]:
nytestdata = testdata.filter(F.col("identifikator") != "id4").drop("numbvar")
compare_dimdf(testdata, nytestdata)

# Test av funksjon compare_columns

In [ ]:
compare_columns(testdata, testdata)

In [ ]:
nytestdata = testdata.drop("numbvar")
compare_columns(testdata, nytestdata)

# Test av funksjon compare_df

In [ ]:
compare_df(testdata, testdata)

In [ ]:
nytestdata = testdata.withColumn(
    "numbvar",
    F.when(F.col("identifikator") == "id3", F.lit(100)).otherwise(F.col("numbvar")),
)
compare_df(testdata, nytestdata)